In [53]:
%matplotlib notebook
# !git pull
# import os;os.environ['LAMBDA_PARQUET_TICK_DB']=rf'X:\lambda_data'


# Generation
Is generating running a backtest where the replay buffer is also going to save prices the time_horizon_seconds later to try to get a dataframe to get most important features.
Private  states are always included(inventory + score) so in the backtest the trades are not changing the results

In [54]:
from notebooks.lambda_imports import *

In [55]:
columns_to_take_pct = 0.35
time_horizon_seconds=5
##
tick = TickDB()
instrument_pk='btcusdt_binance'
start_date=datetime.datetime(year=2020, day=7, month=12)
end_date=datetime.datetime(year=2020, day=7, month=12)
MARKET_MIDPRICE_RELATIVE=True
private_horizon_ticks=5#always included
market_horizon_ticks=10
candle_horizon=5

In [56]:
from mlfinlab.feature_importance import single_feature_importance, mean_decrease_impurity, mean_decrease_accuracy, \
    plot_feature_importance, get_orthogonal_features, feature_pca_analysis


In [57]:
FILENAME = (BACKTEST_OUTPUT_PATH + '/ReplayBuffer_feature_importance_candles.csv')
session_filename='feature_importance.db'
GENERATE_FILE=True

In [58]:
# dill.load_session(session_filename)
depth_data = tick.get_depth(instrument_pk=instrument_pk,start_date=start_date,end_date=end_date)
trade_data = tick.get_trades(instrument_pk=instrument_pk,start_date=start_date,end_date=end_date)
candle_data = tick.get_candles_time(instrument_pk=instrument_pk,start_date=start_date,end_date=end_date)
candle_data.columns = ['_'.join(col).strip() for col in candle_data.columns.values]

downloading btcusdt_binance depth from 20201207 to 20201207
downloading btcusdt_binance trade from 20201207 to 20201207
downloading btcusdt_binance trade from 20201207 to 20201207


In [59]:
columns_states = []

private_states=[]
market__depth_states = []
market__trade_states = []
candle_states = []
#   [inventory_2, inventory_1, inventory_0,
#   score_2, score_1, score_0,
for private_state_horizon in range(private_horizon_ticks-1,-1,-1):
    private_states.append('inventory_%d'%private_state_horizon)
    private_states.append('score_%d'%private_state_horizon)
    
for market_state_horizon in range(market_horizon_ticks-1,-1,-1):
    market__depth_states.append('bid_price_%d'%market_state_horizon)
    market__depth_states.append('ask_price_%d'%market_state_horizon)
    market__depth_states.append('bid_qty_%d'%market_state_horizon)
    market__depth_states.append('ask_qty_%d'%market_state_horizon)
    market__depth_states.append('spread_%d'%market_state_horizon)
    market__depth_states.append('midprice_%d'%market_state_horizon)
    market__depth_states.append('microprice_%d'%market_state_horizon)
    market__trade_states.append('last_close_price_%d'%market_state_horizon)
    market__trade_states.append('last_close_qty_%d'%market_state_horizon)
for candle_state_horizon in range(candle_horizon-1,-1,-1):
  candle_states.append('open_%d'%candle_state_horizon)
  candle_states.append('high_%d'%candle_state_horizon)
  candle_states.append('low_%d'%candle_state_horizon)
  candle_states.append('close_%d'%candle_state_horizon)
candle_states.append('ma')
candle_states.append('std')
candle_states.append('max')
candle_states.append('min')



future_columns = ['future_midprice', 'future_askprice', 'future_bidprice']

columns_states=private_states+market__depth_states+market__trade_states+candle_states
columns_states += future_columns

In [60]:
# MARKET_MIDPRICE_RELATIVE
depth_data['spread']=abs(depth_data['bidPrice0']-depth_data['askPrice0'])
depth_data['midprice']=(depth_data['bidPrice0']+depth_data['askPrice0'])/2
depth_data['microprice']=get_microprice(depth_data)
depth_data['imbalance']=get_imbalance(depth_data,max_depth=5)

if MARKET_MIDPRICE_RELATIVE:
    depth_data['bidPrice0']=abs(depth_data['bidPrice0']-depth_data['midprice'])
    depth_data['askPrice0']=abs(depth_data['askPrice0']-depth_data['midprice'])
    depth_data['microprice']=abs(depth_data['microprice']-depth_data['midprice'])

for lag in range(market_horizon_ticks):
    depth_data['bid_price_%d'%lag]=depth_data['bidPrice0'].shift(lag)
    depth_data['ask_price_%d'%lag]=depth_data['askPrice0'].shift(lag)
    depth_data['bid_qty_%d'%lag]=depth_data['bidQuantity0'].shift(lag)
    depth_data['ask_qty_%d'%lag]=depth_data['askQuantity0'].shift(lag)
    depth_data['spread_%d'%lag]=depth_data['spread'].shift(lag)
    depth_data['midprice_%d'%lag]=depth_data['midprice'].shift(lag)
    depth_data['microprice_%d'%lag]=depth_data['microprice'].shift(lag)

depth_data.tail()

,askPrice0,askPrice1,askPrice2,askPrice3,askPrice4,askQuantity0,askQuantity1,askQuantity2,askQuantity3,askQuantity4,bidPrice0,bidPrice1,bidPrice2,bidPrice3,bidPrice4,bidQuantity0,bidQuantity1,bidQuantity2,bidQuantity3,bidQuantity4,spread,midprice,microprice,imbalance,bid_price_0,...,ask_qty_6,spread_6,midprice_6,microprice_6,bid_price_7,ask_price_7,bid_qty_7,ask_qty_7,spread_7,midprice_7,microprice_7,bid_price_8,ask_price_8,bid_qty_8,ask_qty_8,spread_8,midprice_8,microprice_8,bid_price_9,ask_price_9,bid_qty_9,ask_qty_9,spread_9,midprice_9,microprice_9
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-12-07 22:59:52.856,0.005,19114.79,19114.98,19114.99,19115.68,0.964469,0.097590,0.773956,0.026157,0.061266,0.005,19113.95,19113.84,19112.95,19110.11,10.225415,0.000575,4.738066,0.075924,0.200000,0.01,19114.035,0.004989,0.827618,0.005,...,12.940981,0.01,19113.845,0.003344,0.005,0.005,5.799824,4.104273,0.01,19113.845,0.000873,0.005,0.005,6.286044,4.469543,0.01,19113.845,0.000686,0.005,0.005,5.714493,2.114133,0.01,19113.865,0.004958
2020-12-07 22:59:53.856,0.005,19114.22,19114.98,19117.88,19118.52,3.230739,0.137693,1.324950,1.748000,0.017000,0.005,19113.93,19113.91,19113.85,19113.84,9.648370,0.000575,0.000575,0.000693,4.676666,0.01,19114.035,0.004994,0.498298,0.005,...,3.322172,0.01,19113.845,0.003328,0.005,0.005,5.834448,12.940981,0.01,19113.845,0.003344,0.005,0.005,5.799824,4.104273,0.01,19113.845,0.000873,0.005,0.005,6.286044,4.469543,0.01,19113.845,0.000686
2020-12-07 22:59:54.856,0.005,19114.98,19114.99,19116.28,19117.90,1.367300,0.141252,0.026157,0.103000,0.114279,0.005,19113.84,19113.10,19109.83,19109.70,9.666863,4.677241,0.000575,0.050000,0.070234,0.01,19114.035,0.003321,0.752170,0.005,...,2.240768,0.01,19114.985,0.004159,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,5.834448,12.940981,0.01,19113.845,0.003344,0.005,0.005,5.799824,4.104273,0.01,19113.845,0.000873
2020-12-07 22:59:56.856,0.005,19114.23,19114.98,19114.99,19117.17,1.659582,0.000043,0.141252,0.953784,0.104000,0.005,19114.03,19111.62,19109.82,19103.55,5.298464,0.005500,0.013081,0.800000,0.125000,0.01,19114.175,0.004044,0.522975,0.005,...,2.431178,0.01,19114.985,0.001064,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,5.834448,12.940981,0.01,19113.845,0.003344
2020-12-07 22:59:57.857,0.005,19114.99,19117.27,19118.46,19120.15,0.918936,0.026157,0.114279,0.050000,0.723000,0.005,19110.49,19110.11,19108.82,19108.44,2.824490,0.050000,0.200000,0.800000,1.617000,0.01,19114.485,0.004663,0.509040,0.005,...,2.284575,0.01,19114.595,0.004948,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328


In [61]:
#add future
future_df=pd.DataFrame(columns=future_columns)
future_df['future_midprice']=depth_data['midprice'].shift(periods=-time_horizon_seconds,freq="S")
future_df['future_askprice']=depth_data['askPrice0'].shift(periods=-time_horizon_seconds,freq="S")
future_df['future_bidprice']=depth_data['bidPrice0'].shift(periods=-time_horizon_seconds,freq="S")

In [62]:
for lag in range(market_horizon_ticks):
    trade_data['last_close_price_%d'%lag]=trade_data['price'].shift(lag)
    trade_data['last_close_qty_%d'%lag]=trade_data['quantity'].shift(lag)

In [63]:
if MARKET_MIDPRICE_RELATIVE:    
    candle_data['price_close']=(candle_data['price_close']/candle_data['price_open'])-1.0
    candle_data['price_low']=(candle_data['price_low']/candle_data['price_open'])-1.
    candle_data['price_high']=(candle_data['price_high']/candle_data['price_open'])-1.
    candle_data['price_open']=0.
    
for lag in range(candle_horizon):
    candle_data['open_%d'%lag]=candle_data['price_open'].shift(lag)
    candle_data['close_%d'%lag]=candle_data['price_close'].shift(lag)
    candle_data['low_%d'%lag]=candle_data['price_low'].shift(lag)
    candle_data['high_%d'%lag]=candle_data['price_high'].shift(lag)

candle_data['ma'] = candle_data['price_close'].rolling(window=candle_horizon).mean()
candle_data['std'] = candle_data['price_close'].rolling(window=candle_horizon).std()
candle_data['max'] = candle_data['price_high'].rolling(window=candle_horizon).max()
candle_data['min'] = candle_data['price_low'].rolling(window=candle_horizon).min()

In [64]:
depth_data_2 = depth_data[market__depth_states].reset_index()
trade_data_2 = trade_data[market__trade_states].reset_index()
candle_data_2 = candle_data[candle_states].reset_index()
future_data_2 = future_df.reset_index()

output_data = pd.concat([depth_data_2, trade_data_2,candle_data_2,future_data_2])
output_data.fillna(method='ffill',inplace=True)
output_data.dropna(inplace=True)
output_data.set_index('date',inplace=True)
output_data.sort_index(inplace=True)
output_data.head()


,bid_price_9,ask_price_9,bid_qty_9,ask_qty_9,spread_9,midprice_9,microprice_9,bid_price_8,ask_price_8,bid_qty_8,ask_qty_8,spread_8,midprice_8,microprice_8,bid_price_7,ask_price_7,bid_qty_7,ask_qty_7,spread_7,midprice_7,microprice_7,bid_price_6,ask_price_6,bid_qty_6,ask_qty_6,...,low_4,close_4,open_3,high_3,low_3,close_3,open_2,high_2,low_2,close_2,open_1,high_1,low_1,close_1,open_0,high_0,low_0,close_0,ma,std,max,min,future_midprice,future_askprice,future_bidprice
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-12-06 22:59:48.342,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493,19114.49,19109.01,19129.065,2.055,2.055
2020-12-06 22:59:49.342,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493,19114.49,19109.01,19129.735,0.005,0.005
2020-12-06 22:59:51.342,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493,19114.49,19109.01,19129.735,0.005,0.005
2020-12-06 22:59:52.342,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493,19114.49,19109.01,19129.715,0.005,0.005
2020-12-06 22:59:53.343,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493,19114.49,19109.01,19129.695,0.005,0.005


In [65]:
if GENERATE_FILE:
    output_data.to_csv(FILENAME)

In [66]:
output_data.columns

Index(['bid_price_9', 'ask_price_9', 'bid_qty_9', 'ask_qty_9', 'spread_9',
       'midprice_9', 'microprice_9', 'bid_price_8', 'ask_price_8', 'bid_qty_8',
       ...
       'high_0', 'low_0', 'close_0', 'ma', 'std', 'max', 'min',
       'future_midprice', 'future_askprice', 'future_bidprice'],
      dtype='object', length=117)

# Feature Importance

In [67]:
filename = FILENAME  # (models_path + '/ReplayBufferPrinterAlgo_ethbtc_replaybuffer.csv')
if not GENERATE_FILE:
    output_data = pd.read_csv(FILENAME, index_col=0)
output_data.describe()
df=output_data

In [68]:
dill.dump_session(session_filename)


ValueError: ctypes objects containing pointers cannot be pickled

127

In [77]:

assert len(output_data.columns)==len(columns_states)-len(private_states)
output_data.columns

Index(['bid_price_9', 'ask_price_9', 'bid_qty_9', 'ask_qty_9', 'spread_9',
       'midprice_9', 'microprice_9', 'bid_price_8', 'ask_price_8', 'bid_qty_8',
       ...
       'high_0', 'low_0', 'close_0', 'ma', 'std', 'max', 'min',
       'future_midprice', 'future_askprice', 'future_bidprice'],
      dtype='object', length=117)

In [79]:
output_data['midprice_diff_future'] = output_data['future_midprice'] - output_data['midprice_0']
output_data['target'] = output_data['midprice_diff_future'] > 0

## Private

In [ ]:
# df[state.get_private_columns()].describe()

## Market

In [81]:
df[market__depth_states+market__trade_states].describe()

,bid_price_9,ask_price_9,bid_qty_9,ask_qty_9,spread_9,midprice_9,microprice_9,bid_price_8,ask_price_8,bid_qty_8,ask_qty_8,spread_8,midprice_8,microprice_8,bid_price_7,ask_price_7,bid_qty_7,ask_qty_7,spread_7,midprice_7,microprice_7,bid_price_6,ask_price_6,bid_qty_6,ask_qty_6,...,bid_qty_0,ask_qty_0,spread_0,midprice_0,microprice_0,last_close_price_9,last_close_qty_9,last_close_price_8,last_close_qty_8,last_close_price_7,last_close_qty_7,last_close_price_6,last_close_qty_6,last_close_price_5,last_close_qty_5,last_close_price_4,last_close_qty_4,last_close_price_3,last_close_qty_3,last_close_price_2,last_close_qty_2,last_close_price_1,last_close_qty_1,last_close_price_0,last_close_qty_0
count,81859.000,81859.000,8.185900e+04,8.185900e+04,81859.00,81859.000,8.185900e+04,81859.000,81859.000,8.185900e+04,8.185900e+04,81859.00,8.185900e+04,81859.000000,81859.000,81859.000,8.185900e+04,8.185900e+04,81859.00,8.185900e+04,81859.000000,81859.000,81859.000,81859.0000,81859.000000,...,81859.00000,8.185900e+04,81859.00,8.185900e+04,81859.000000,8.185900e+04,8.185900e+04,8.185900e+04,8.185900e+04,8.185900e+04,8.185900e+04,8.185900e+04,8.185900e+04,8.185900e+04,8.185900e+04,8.185900e+04,81859.00000,8.185900e+04,8.185900e+04,8.185900e+04,8.185900e+04,8.185900e+04,8.185900e+04,8.185900e+04,81859.000000
mean,0.005,0.005,6.333297e+00,3.322172e+00,0.01,19113.845,3.327818e-03,0.005,0.005,2.661987e+00,2.240768e+00,0.01,1.911498e+04,0.004159,0.005,0.005,1.105365e+00,2.431178e+00,0.01,1.911498e+04,0.001064,0.005,0.005,0.4665,2.284575,...,2.82449,9.189360e-01,0.01,1.911448e+04,0.004663,1.911418e+04,5.478000e-03,1.911423e+04,5.457000e-03,1.911418e+04,4.721900e-02,1.911418e+04,3.544000e-03,1.911448e+04,1.454000e-03,1.911449e+04,0.00548,1.911449e+04,5.734000e-03,1.911449e+04,5.734000e-03,1.911449e+04,4.181000e-03,1.911449e+04,0.001077
std,0.000,0.000,8.881838e-16,1.332276e-15,0.00,0.000,4.336835e-19,0.000,0.000,4.440919e-16,8.881838e-16,0.00,1.091400e-11,0.000000,0.000,0.000,4.440919e-16,4.440919e-16,0.00,1.091400e-11,0.000000,0.000,0.000,0.0000,0.000000,...,0.00000,2.220460e-16,0.00,1.091400e-11,0.000000,3.638001e-12,2.602101e-18,3.638001e-12,2.602101e-18,3.638001e-12,6.938936e-18,3.638001e-12,4.336835e-19,3.638001e-12,2.168418e-19,7.276002e-12,0.00000,7.276002e-12,1.734734e-18,7.276002e-12,1.734734e-18,7.276002e-12,1.734734e-18,7.276002e-12,0.000000
min,0.005,0.005,6.333297e+00,3.322172e+00,0.01,19113.845,3.327818e-03,0.005,0.005,2.661987e+00,2.240768e+00,0.01,1.911499e+04,0.004159,0.005,0.005,1.105365e+00,2.431178e+00,0.01,1.911499e+04,0.001064,0.005,0.005,0.4665,2.284575,...,2.82449,9.189360e-01,0.01,1.911449e+04,0.004663,1.911418e+04,5.478000e-03,1.911423e+04,5.457000e-03,1.911418e+04,4.721900e-02,1.911418e+04,3.544000e-03,1.911448e+04,1.454000e-03,1.911449e+04,0.00548,1.911449e+04,5.734000e-03,1.911449e+04,5.734000e-03,1.911449e+04,4.181000e-03,1.911449e+04,0.001077
25%,0.005,0.005,6.333297e+00,3.322172e+00,0.01,19113.845,3.327818e-03,0.005,0.005,2.661987e+00,2.240768e+00,0.01,1.911499e+04,0.004159,0.005,0.005,1.105365e+00,2.431178e+00,0.01,1.911499e+04,0.001064,0.005,0.005,0.4665,2.284575,...,2.82449,9.189360e-01,0.01,1.911449e+04,0.004663,1.911418e+04,5.478000e-03,1.911423e+04,5.457000e-03,1.911418e+04,4.721900e-02,1.911418e+04,3.544000e-03,1.911448e+04,1.454000e-03,1.911449e+04,0.00548,1.911449e+04,5.734000e-03,1.911449e+04,5.734000e-03,1.911449e+04,4.181000e-03,1.911449e+04,0.001077
50%,0.005,0.005,6.333297e+00,3.322172e+00,0.01,19113.845,3.327818e-03,0.005,0.005,2.661987e+00,2.240768e+00,0.01,1.911499e+04,0.004159,0.005,0.005,1.105365e+00,2.431178e+00,0.01,1.911499e+04,0.001064,0.005,0.005,0.4665,2.284575,...,2.82449,9.189360e-01,0.01,1.911449e+04,0.004663,1.911418e+04,5.478000e-03,1.911423e+04,5.457000e-03,1.911418e+04,4.721900e-02,1.911418e+04,3.544000e-03,1.911448e+04,1.454000e-03,1.911449e+04,0.00548,1.911449e+04,5.734000e-03,1.911449e+04,5.734000e-03,1.911449e+04,4.181000e-03,1.911449e+04,0.001077
75%,0.005,0.005,6.333297e+00,3.322172e+00,

## Candles

In [82]:
df[candle_states].describe()

,open_4,high_4,low_4,close_4,open_3,high_3,low_3,close_3,open_2,high_2,low_2,close_2,open_1,high_1,low_1,close_1,open_0,high_0,low_0,close_0,ma,std,max,min
count,8.185900e+04,81859.0,8.185900e+04,8.185900e+04,8.185900e+04,81859.0,8.185900e+04,81859.0,81859.0,8.185900e+04,81859.00,8.185900e+04,8.185900e+04,81859.0,8.185900e+04,81859.00,81859.00,8.185900e+04,8.185900e+04,8.185900e+04,81859.00,8.185900e+04,8.185900e+04,8.185900e+04
mean,1.910830e+04,19110.0,1.910268e+04,1.910901e+04,1.910901e+04,19110.0,1.910377e+04,19110.0,19110.0,1.911318e+04,19109.69,1.911161e+04,1.911182e+04,19115.0,1.910993e+04,19112.09,19112.09,1.911499e+04,1.911204e+04,1.911449e+04,19111.44,2.105493e+00,1.911449e+04,1.910901e+04
std,7.276002e-12,0.0,3.638001e-12,7.276002e-12,7.276002e-12,0.0,3.638001e-12,0.0,0.0,3.638001e-12,0.00,1.091400e-11,3.638001e-12,0.0,3.638001e-12,0.00,0.00,7.276002e-12,3.638001e-12,7.276002e-12,0.00,4.440919e-16,7.276002e-12,7.276002e-12
min,1.910830e+04,19110.0,1.910268e+04,1.910901e+04,1.910901e+04,19110.0,1.910377e+04,19110.0,19110.0,1.911318e+04,19109.69,1.911161e+04,1.911182e+04,19115.0,1.910993e+04,19112.09,19112.09,1.911499e+04,1.911204e+04,1.911449e+04,19111.44,2.105493e+00,1.911449e+04,1.910901e+04
25%,1.910830e+04,19110.0,1.910268e+04,1.910901e+04,1.910901e+04,19110.0,1.910377e+04,19110.0,19110.0,1.911318e+04,19109.69,1.911161e+04,1.911182e+04,19115.0,1.910993e+04,19112.09,19112.09,1.911499e+04,1.911204e+04,1.911449e+04,19111.44,2.105493e+00,1.911449e+04,1.910901e+04
50%,1.910830e+04,19110.0,1.910268e+04,1.910901e+04,1.910901e+04,19110.0,1.910377e+04,19110.0,19110.0,1.911318e+04,19109.69,1.911161e+04,1.911182e+04,19115.0,1.910993e+04,19112.09,19112.09,1.911499e+04,1.911204e+04,1.911449e+04,19111.44,2.105493e+00,1.911449e+04,1.910901e+04
75%,1.910830e+04,19110.0,1.910268e+04,1.910901e+04,1.910901e+04,19110.0,1.910377e+04,19110.0,19110.0,1.911318e+04,19109.69,1.911161e+04,1.911182e+04,19115.0,1.910993e+04,19112.09,19112.09,1.911499e+04,1.911204e+04,1.911449e+04,19111.44,2.105493e+00,1.911449e+04,1.910901e+04
max,1.910830e+04,19110.0,1.910268e+04,1.910901e+04,1.910901e+04,19110.0,1.910377e+04,19110.0,19110.0,1.911318e+04,19109.69,1.911161e+04,1.911182e+04,19115.0,1.910993e+04,19112.09,19112.09,1.911499e+04,1.911204e+04,1.911449e+04,19111.44,2.105493e+00,1.911449e+04,1.910901e+04


In [83]:
columns_X = list(df.columns[:-7])
X_input = df[columns_X]
y_input = df['target']

In [84]:
# %%
from sklearn.model_selection import KFold
cv_gen = KFold(n_splits=4)

from sklearn.ensemble import RandomForestClassifier

base_est = RandomForestClassifier(n_estimators=1, criterion='entropy', bootstrap=False,
                                  class_weight='balanced_subsample')

# pca_orthog = PCA(n_components=200)

# X_train_orthog = pd.DataFrame(pca_orthog.fit_transform(X_train),index=X_train.index)
X_train_orthog = X_input
y_train = y_input


total_columns = X_train_orthog.shape[1]

columns_to_take_mdi = int(total_columns*columns_to_take_pct*0.25)
columns_to_take_mda = int(total_columns*columns_to_take_pct*0.25)
columns_to_take_sfi = int(total_columns*columns_to_take_pct*0.5)

In [85]:
print('Initially we have %d columns_states that we are going to reduce to a maximun of %d columns'%(X_input.shape[1],columns_to_take_mdi+columns_to_take_mda+columns_to_take_sfi))

Initially we have 112 columns_states that we are going to reduce to a maximun of 37 columns


In [86]:
X_train_orthog=X_train_orthog.replace([np.inf, -np.inf], np.nan)
X_train_orthog=X_train_orthog.dropna()
X_train_orthog

,bid_price_9,ask_price_9,bid_qty_9,ask_qty_9,spread_9,midprice_9,microprice_9,bid_price_8,ask_price_8,bid_qty_8,ask_qty_8,spread_8,midprice_8,microprice_8,bid_price_7,ask_price_7,bid_qty_7,ask_qty_7,spread_7,midprice_7,microprice_7,bid_price_6,ask_price_6,bid_qty_6,ask_qty_6,...,last_close_qty_1,last_close_price_0,last_close_qty_0,open_4,high_4,low_4,close_4,open_3,high_3,low_3,close_3,open_2,high_2,low_2,close_2,open_1,high_1,low_1,close_1,open_0,high_0,low_0,close_0,ma,std
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-12-06 22:59:48.342,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
2020-12-06 22:59:49.342,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
2020-12-06 22:59:51.342,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
2020-12-06 22:59:52.342,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
2020-12-06 22:59:53.343,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-07 22:59:47.856,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
2020-12-07 22:59:48.856,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
2020-12-07 22:59:49.856,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005

In [87]:
X_train_orthog.replace([np.inf, -np.inf], np.nan)

,bid_price_9,ask_price_9,bid_qty_9,ask_qty_9,spread_9,midprice_9,microprice_9,bid_price_8,ask_price_8,bid_qty_8,ask_qty_8,spread_8,midprice_8,microprice_8,bid_price_7,ask_price_7,bid_qty_7,ask_qty_7,spread_7,midprice_7,microprice_7,bid_price_6,ask_price_6,bid_qty_6,ask_qty_6,...,last_close_qty_1,last_close_price_0,last_close_qty_0,open_4,high_4,low_4,close_4,open_3,high_3,low_3,close_3,open_2,high_2,low_2,close_2,open_1,high_1,low_1,close_1,open_0,high_0,low_0,close_0,ma,std
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-12-06 22:59:48.342,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
2020-12-06 22:59:49.342,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
2020-12-06 22:59:51.342,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
2020-12-06 22:59:52.342,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
2020-12-06 22:59:53.343,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-07 22:59:47.856,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
2020-12-07 22:59:48.856,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005,0.4665,2.284575,...,0.004181,19114.49,0.001077,19108.3,19110.0,19102.68,19109.01,19109.01,19110.0,19103.77,19110.0,19110.0,19113.18,19109.69,19111.61,19111.82,19115.0,19109.93,19112.09,19112.09,19114.99,19112.04,19114.49,19111.44,2.105493
2020-12-07 22:59:49.856,0.005,0.005,6.333297,3.322172,0.01,19113.845,0.003328,0.005,0.005,2.661987,2.240768,0.01,19114.985,0.004159,0.005,0.005,1.105365,2.431178,0.01,19114.985,0.001064,0.005,0.005

In [88]:
y_input

date
2020-12-06 22:59:48.342     True
2020-12-06 22:59:49.342     True
2020-12-06 22:59:51.342     True
2020-12-06 22:59:52.342     True
2020-12-06 22:59:53.343     True
                           ...  
2020-12-07 22:59:47.856    False
2020-12-07 22:59:48.856    False
2020-12-07 22:59:49.856    False
2020-12-07 22:59:51.856    False
2020-12-07 22:59:52.857    False
Name: target, Length: 81859, dtype: bool

In [89]:
# %% MDI
# This score can be obtained from tree-based classifiers and corresponds to sklearn’s feature_importances attribute. MDI uses in-sample (IS) performance to estimate feature importance.
print('MDI starting...')
base_est.fit(X_train_orthog, y_input)

mdi_feature_imp = mean_decrease_impurity(base_est, X_train_orthog.columns)
mdi_best_columns = mdi_feature_imp.sort_values(by='mean', ascending=False)[:columns_to_take_mdi].index
mdi_best_columns = list(mdi_best_columns.values)
print(mdi_best_columns)

MDI starting...
['bid_price_9', 'ask_price_9', 'bid_qty_9', 'ask_qty_9', 'spread_9', 'midprice_9', 'microprice_9', 'bid_price_8', 'ask_price_8']


In [90]:
def plot_feature_importance(importance,names,model_type,columns_to_take):
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df=fi_df.sort_values(by=['feature_importance'], ascending=False).iloc[:columns_to_take]

    #Define size of bar plot
    plt.figure(figsize=(12,10))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + ' feature importance')
    plt.xlabel('feature importance')
    plt.ylabel('features')
    
plot_feature_importance(base_est.feature_importances_, X_train_orthog.columns,'MDI',columns_to_take_mdi*2)

<IPython.core.display.Javascript object>

In [ ]:
# from sklearn.tree import export_graphviz
# # Export as dot file
# export_graphviz(base_est.estimators_[0], out_file='tree.dot', 
#                 feature_names = base_est.feature_importances_,
#                 class_names = X_train_orthog.columns,
#                 rounded = True, proportion = False, 
#                 precision = 2, filled = True)

In [91]:
# %% MDA
# This method can be applied to any classifier, not only tree based. MDA uses out-of-sample (OOS) performance in order to estimate feature importance.
from sklearn.metrics import accuracy_score, log_loss

print('MDA starting...')
mda_feature_imp = mean_decrease_accuracy(base_est, X_train_orthog, y_input, cv_gen, scoring=log_loss)
mda_best_columns = mda_feature_imp.sort_values(by='mean', ascending=False)[:columns_to_take_mda].index
mda_best_columns = list(mda_best_columns.values)
print(mda_best_columns)


MDA starting...
['bid_price_9', 'ask_price_9', 'last_close_price_3', 'last_close_qty_4', 'last_close_price_4', 'last_close_qty_5', 'last_close_price_5', 'last_close_qty_6', 'last_close_price_6']


In [ ]:
plot_feature_importance(base_est.feature_importances_, X_train_orthog.columns,'MDA',columns_to_take_mda*2)

In [ ]:
# %% SFI
# MDA and MDI feature suffer from substitution effects. If two features are highly correlated,
# one of them will be considered as important while the other one will be redundant.
# SFI is a OOS feature importance estimator which doesn’t suffer from substitution effects because it estimates each feature importance separately.
print('SFI starting...')
sfi_feature_imp = single_feature_importance(base_est, X_train_orthog, y_input, cv_gen, scoring=accuracy_score)
sfi_best_columns = sfi_feature_imp.sort_values(by='mean', ascending=False)[:columns_to_take_sfi].index
sfi_best_columns = list(sfi_best_columns.values)
print(sfi_best_columns)

In [ ]:
# plot_feature_importance(base_est.feature_importances_, X_train_orthog.columns,'SFI',columns_to_take_sfi*2)

In [ ]:
private_state_columns=state.get_private_columns()
private_state_columns

In [ ]:

# selection_columns = set(private_state_columns+mda_best_columns + sfi_best_columns + mdi_best_columns)
selection_columns = set(mda_best_columns + sfi_best_columns + mdi_best_columns)
print('feature_importance_input finished to %d columns' % len(selection_columns))

In [ ]:
selection_columns

In [ ]:
set(list(selection_columns)+list(private_state_columns))

In [ ]:
dill.dump_session(session_filename)